In [ ]:
from LDAR_company import LDAR_travel
import datetime
import numpy as np
import random
import os 
import pandas as pd 

In [ ]:
path = r"C:\Users\cookg\Desktop\LDAR_Travel_Sim\AreaofRegion"

In [ ]:
listnumbers = [0,1,2,3,4,5,6,7,8,9]
Folders = ['Grid{}'.format(e) for e in listnumbers]
for f in Folders:
    os.makedirs(os.path.join(path, f))
    
Folders = ['Forest{}'.format(e) for e in listnumbers]
for f in Folders:
    os.makedirs(os.path.join(path, f))

In [ ]:
A = LDAR_travel('shared',1,26912,'mirror')
X = [0]
Y = [0]
A.create_empty_graph_with_sc(X,Y)
A.create_random_points_as_LDAR_sites(299,10000)
nid,eid = A.create_highways(2138027,676423396,10000,5000,80)
nid,eid = A.create_backroads_grid(nid,eid,80,30)
A.create_GDB('Agriculture',4326)
A.create_plot(setting = "Agriculture")

In [ ]:
Areas = [10000,15000,20000,25000,30000,35000,40000]

In [ ]:
reps = [0,1,2,3,4,5,6,7,8,9]

In [ ]:
for rep in reps:
    for ran in Areas:
        A = LDAR_travel('shared',1,26912,'mirror')
        X = [0]
        Y = [0]
        A.create_empty_graph_with_sc(X,Y)
        nid,eid = A.create_highways(2138027,676423396,ran,5000,80)
        A.create_random_points_as_LDAR_sites(299,ran)
        A.sites['lon'] = A.sites.geometry.x
        A.sites['lat'] = A.sites.geometry.y
        A.sites['dist_diff'] = np.zeros(len(A.sites))
        A.sites['time_diff'] = np.zeros(len(A.sites))
        A.sites['siteID'] = np.arange(1,len(A.sites)+1,1)
        nopt = 5 
        sn = 300
        P = list(range(nopt))
        OPT = [random.choice(P) for i in range(sn)]

        while len(set(OPT)) != nopt: 
            OPT = [random.choice(P) for i in range(sn)]

        A.sites['OPT'] = OPT 

        A.sample_inspection_time(r'D:\Shared_service_LDAR\ins_time.csv')
        A.sites.to_csv(r"D:\Shared_service_LDAR\CVT_mins\AreaofRegion\sites_{}_{}.csv".format(rep,ran),sep=',')
    
    
    

In [ ]:
Rep = [0,1,2,3,4,5,6,7,8,9]

### Grid 

In [ ]:
Areas = [10000,15000,20000,25000,30000,35000,40000]
Rep = [0,1,2,3,4,5,6,7,8,9]

In [ ]:
for rep in Rep:
    RL = [] 
    for area in Areas:
        A = LDAR_travel('shared',1,26912,'mirror')
        A.read_site_list(r"D:\Shared_service_LDAR\CVT_mins\AreaofRegion\sites_{}_{}.csv".format(rep,area),"lon",'lat',prj=26912)
        X = [0]
        Y = [0]
        A.create_empty_graph_with_sc(X,Y)
        nid,eid = A.create_highways(2138027,676423396,area,5000,80)
        nid,eid = A.create_backroads_grid(nid,eid,80,30)
        A.create_GDB('Agriculture',4326)
        
        road_length = sum(A.BR.length) + sum(A.highways.length)
        RL.append(road_length)
        
        A.sites['node'] = A.Site_nodes
        
       ####################### Shared ########################
        start_time = datetime.datetime(2022,1,1,8)
        end_time = datetime.datetime(2022,1,1,16)
        A.create_agent(1,start_time,end_time)
        A.deploy_agents()
        A.generate_inspection_report()
        i = 0 
        for report in A.Report: 
            report.to_csv(r"D:\Shared_service_LDAR\CVT_mins\AreaofRegion\Grid{}\results_share_{}_{}.csv".format(rep,area,i),sep=',')

            i += 1 

        ####################### Duty-holder-Independent#########
        start_time = datetime.datetime(2022,1,1,8)
        end_time = datetime.datetime(2022,1,1,16)
        A.create_agent(1,start_time,end_time,Shared = False, Dutyholder = 'OPT')

        A.deploy_agents(Shared = False)
        A.generate_inspection_report(Shared = False)

        i = 0 
        for report in A.Report: 
            report.to_csv(r"D:\Shared_service_LDAR\CVT_mins\AreaofRegion\Grid{}\results_opt_{}_{}.csv".format(rep,area,i),sep=',')

            i += 1 da
            
    df1 = pd.DataFrame(data = {'road_length':RL,
                              'Areas':Areas})
    
    df1.to_csv(r"D:\Shared_service_LDAR\CVT_mins\AreaofRegion\Grid{}\road_length.csv".format(rep),sep=',')

### Forest

In [ ]:
R = [0,1,2,3,4,5,6,7,8,9]
Areas = [10000,15000,20000,25000,30000,35000,40000]

In [ ]:
for rep in R:
    RL = [] 
    for area in Areas:
        A = LDAR_travel('shared',1,26912,'mirror')
        A.read_site_list(r"D:\Shared_service_LDAR\CVT_mins\AreaofRegion\sites_{}_{}.csv".format(rep,area),"lon",'lat',prj=26912)
        X = [0]
        Y = [0]
        A.create_empty_graph_with_sc(X,Y)
        nid,eid = A.create_highways(2138027,676423396,area,5000,80,setting= "Forested")
        nid,eid = A.create_gravel_roads_forest (area,500,10,nid,eid,80,50)
        nid,eid = A.create_backroads_forest(nid,eid,50,30)
        A.create_GDB('Forested',4326)
        
        road_length = sum(A.BR.length) + sum(A.highways.length) + sum(A.gravelroads.length)
        RL.append(road_length)
        
        A.sites['node'] = A.Site_nodes
        
       ####################### Shared ########################
        start_time = datetime.datetime(2022,1,1,8)
        end_time = datetime.datetime(2022,1,1,16)
        A.create_agent(1,start_time,end_time)
        A.deploy_agents()
        A.generate_inspection_report()
        i = 0 
        for report in A.Report: 
            report.to_csv(r"D:\Shared_service_LDAR\CVT_mins\AreaofRegion\Forest{}\results_share_{}_{}.csv".format(rep,area,i),sep=',')

            i += 1 

        ####################### Duty-holder-Independent#########
        start_time = datetime.datetime(2022,1,1,8)
        end_time = datetime.datetime(2022,1,1,16)
        A.create_agent(1,start_time,end_time,Shared = False, Dutyholder = 'OPT')

        A.deploy_agents(Shared = False)
        A.generate_inspection_report(Shared = False)

        i = 0 
        for report in A.Report: 
            report.to_csv(r"D:\Shared_service_LDAR\CVT_mins\AreaofRegion\Forest{}\results_opt_{}_{}.csv".format(rep,area,i),sep=',')

            i += 1 
            
    df1 = pd.DataFrame(data = {'road_length':RL,
                              'Areas':Areas})
    
    df1.to_csv(r"D:\Shared_service_LDAR\CVT_mins\AreaofRegion\Forest{}\road_length.csv".format(rep),sep=',')